In [ ]:
import math
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import matplotlib.pyplot as plt
from itertools import islice

from torchlensmaker.raytracing import super_refraction, clamped_refraction, ray_point_squared_distance, refraction
from torchlensmaker.optics import FocalPointLoss, ParallelBeamUniform, ParallelBeamRandom, FixedGap, RefractiveSurface, OpticalStack
from torchlensmaker.shapes import Parabola, PiecewiseLine, Line, CircularArc, BezierSpline
from torchlensmaker.training import render, optimize

In [ ]:

lens_radius = 25. # lens radius in mm
surface = PiecewiseLine(lens_radius, init=[(lens_radius/4, lens_radius/2, 3/4*lens_radius, lens_radius), (-1.0, -2.0, -3.0, -4.0)])
#surface = CircularArc(lens_radius, [200.])

print(surface.coefficients())
print(surface.evaluate(torch.tensor([1.5])))

optics = OpticalStack([
    ParallelBeamRandom(radius=lens_radius),
    FixedGap(15.),
    
    RefractiveSurface(surface, (1.49, 1.0)),
    
    FixedGap(60.0),
    FocalPointLoss(),
])

render(optics, num_rays=10)


optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-1),
    num_rays = 20,
    num_iter = 100,
)

render(optics, num_rays=10)

